In [1]:
open System
open System.IO
open System.Text.RegularExpressions

In [2]:
let test = 
    """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11"""

In [3]:
type Card = {Id: int; WinningNumbers: int list; ScratchedNumbers: int list}
let createCard id w s = {Id = id; WinningNumbers = w; ScratchedNumbers = s}

In [4]:
#r "nuget:FParsec"
open FParsec

Installed Packages FParsec, 1.1.1

In [5]:
let pId  = pstring "Card" >>. spaces >>. pint32 .>> pstring ":" .>> spaces 
let pWinningNumbers = manyTill (spaces >>. pint32 .>> spaces) (pstring "|")
let pScratchedNumbers = many (spaces >>. pint32 .>> (spaces <|> skipNewline)) 
let lineToCard (line:string) = 
    match run (pipe3 pId pWinningNumbers pScratchedNumbers createCard) line with 
    | Success (card, _, _) -> card 
    | _ -> failwith $"Problems with: {line}"

In [6]:
let points (card : Card) = 
    card.ScratchedNumbers
    |> List.filter (fun x -> List.contains x card.WinningNumbers)
    |> List.length
    |> (+) (-1)
    |> pown 2

In [7]:
test.Split("\n")
|> Array.sumBy (lineToCard >> points)

13

In [8]:
File.ReadAllLines("input_4.txt")
|> Array.sumBy (lineToCard >> points)

23678

# Part 2

In [9]:
let winningNumbers (card: Card) =
    card.ScratchedNumbers
    |> List.filter (fun x -> List.contains x card.WinningNumbers)
    |> List.length

In [10]:
let cards = File.ReadAllLines("input_4.txt") |> Array.map lineToCard

In [11]:
let mutable cardsOfGame = Array.init cards.Length (fun _ -> 1) // in the beginning we have 1 of each

for id in [0..cards.Length-1] do 
    for toCopy in [1..(winningNumbers cards[id])] do
        cardsOfGame[id+toCopy] <-  cardsOfGame[id] + cardsOfGame[id+toCopy] 

In [12]:
Array.sum cardsOfGame

15455663